# Every-visit MC prediction 


 
(c) Fabrice Mulotti

In [1]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict

Create a blackjack environment:

In [2]:
env = gym.make('Blackjack-v1')

## Définissons une politque

state[0] renvoie le nombre de point du joueur.   

L'action :      
0 est standby   
1 est hit (nouvelle carte)  

In [3]:
def policy(pstate):
    return 0 if pstate[0] > 19 else 1

In [4]:
## créons notre environnement
state = env.reset()
print(state)

((13, 6, 0), {})


In [5]:
print(policy(state[0]))

1


In [6]:
# De devrait pas renvoyer d'erreur
assert policy([19,2,2]) == 1

In [7]:
# Devrait renvoyer une erreur
assert policy([20,2,2]) == 1

AssertionError: 

***
## Generatrice d'épisode
Notre prochain fonction va nous permettre de générer des épisodes


In [8]:
def generate_episode(policy,max_turn):
    
    # Enregistrement des épisodes
    episode = []
    
    # Remise à zéro env
    ge_state = env.reset()[0]
  
    # déroulement de la partie
    for i in range(max_turn):
        # choix d'une action en fonction de notre police
        action = policy(ge_state)
        
        # on joue notre action en on récupère les résultats
        ge_next_state, reward, done, truncated, info = env.step(action)
        
        # enregistrement
        episode.append((ge_state, action, reward))
        
        # Condition de fin ?
        if done or truncated:
            break

        # sinon la partie continue
        ge_state = ge_next_state

    return episode

In [15]:
# Devrait produit une partie
generate_episode(policy,100)

[((12, 3, 0), 1, 0.0), ((20, 3, 0), 0, 0.0)]

***
## Calcul de la fonction de valeur

Nous allons devoir générer un grand nombre d'épisode et ajuster la `valeur moyenne de v` à chaque passage sur un état s

 

In [16]:
# total_return = fonction de valeur pour chaque état
total_return = defaultdict(float)

# N nombre de passage sur chaque état
N = defaultdict(int)

In [17]:
# Nombre d'itération
num_iterations = 50000
gamma=0.90

In [18]:
for i in range(num_iterations):
    
    # générons un épisode
    episode = generate_episode(policy,100)

    # stockons l'état, action et récompense obtenue / zip permet de prendre chaque 1, 2eme, 3eme.. terme
    states, actions, rewards = zip(*episode)
    # pour chaque état rencontré dans l'épisode

    G=0
    # print(states)
    for t in range(len(states)-1,-1,-1):
        state=states[t]
        # print(f"t={t}, state={state}, R={R}")
        R=R*gamma+rewards[t]
        # if state not in states[t:]:
        total_return[state] += R
        N[state] += 1
            

***
## Analysons nos résultats
Convertissons les données en dataframe pour une lecture plus facile

In [19]:
total_return = pd.DataFrame(total_return.items(),columns=['state', 'total_return'])

In [20]:
N = pd.DataFrame(N.items(),columns=['state', 'N'])

Merge des deux dataframe

In [21]:
df = pd.merge(total_return, N, on="state")

Résultat : 

In [22]:
df.head(10)

,state,total_return,N
0,"(14, 10, 0)",-1180.45190,1942
1,"(12, 10, 0)",-992.83641,1783
2,"(17, 10, 1)",-108.01080,265
3,"(20, 2, 0)",503.00000,776
4,"(17, 2, 0)",-351.52000,516
5,"(12, 7, 0)",-249.18880,487
6,"(15, 7, 0)",-313.45100,521
7,"(13, 4, 0)",-275.69400,491
8,"(20, 9, 0)",503.00000,679
9,"(10, 9, 0)",-14.44410,187


Calculons la fonction de valeur.
A ce stade nous avons tous les éléments (cumul des G et nombre de passage N)

In [23]:
df['value'] = df['total_return']/df['N']

In [24]:
df.head(20)

,state,total_return,N,value
0,"(14, 10, 0)",-1180.45190,1942,-0.607854
1,"(12, 10, 0)",-992.83641,1783,-0.556835
2,"(17, 10, 1)",-108.01080,265,-0.407588
3,"(20, 2, 0)",503.00000,776,0.648196
4,"(17, 2, 0)",-351.52000,516,-0.681240
5,"(12, 7, 0)",-249.18880,487,-0.511681
6,"(15, 7, 0)",-313.45100,521,-0.601633
7,"(13, 4, 0)",-275.69400,491,-0.561495
8,"(20, 9, 0)",503.00000,679,0.740795
9,"(10, 9, 0)",-14.44410,187,-0.077241


***
## Analyse

A ce stade nous pouvons apprécier la valeur de notre politique (seuil à 19).
Exemple dans une situation ou nous avons 21 points, et par exemple le croupier 9 points 



In [30]:
df[df['state']==(21,10,False)]['value'].values

array([0.90370852])

In [34]:
df[df['state']==(21,10,False)]

,state,total_return,N,value
46,"(21, 10, 0)",1389.0,1537,0.903709


La fonction de valeur est proche de 1. Donc efficiente.

A contrario regardons ce résultat pour une situation ou nous avons 5 points et le croupier 8 points :

In [35]:
df[df['state']==(5,8,False)]['value'].values

array([-0.48265116])

La valeur est moindre car l'incertitude de la suite de la partie plus grande

Impact d'avoir un AS

In [39]:
df[df['state']==(12,10,True)]['value'].values

array([-0.30662707])

In [40]:
df[df['state']==(12,10,False)]['value'].values

array([-0.55683478])